## start Spark

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589822417901_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data_songs = 's3a://udacity-dend/song-data/A/*/*/*.json'
data_logs = 's3a://udacity-dend/log-data/*/*/*.json'

output_data = 's3a://<MY_BUCKET>'
# data = './data/songs/A/*/*.json'
# output_data = './data/output'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = spark.read.json(data_songs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

## extract columns to create songs table

In [23]:
songs_table = df['song_id', 'title', 'artist_id', 'year', 'duration']
songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [22]:
# df.createOrReplaceTempView("staging_songs")
# songs_table = spark.sql("""
#     SELECT song_id, title, artist_id, year, duration
#     FROM staging_songs
#     ORDER BY song_id
# """)  #.collect()

# songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
songs_table.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGXVFI12A6D4FBF6A|Somebody Was Pray...|ARDWI4G1187FB369E1|   0|226.58567|
+------------------+--------------------+------------------+----+---------+

In [26]:
# write songs table to parquet files partitioned by year and artist
s_song_s3_path = f"s3a://{output_data}/songs.parquet"
songs_table.write.mode('overwrite').partitionBy('year', 'artist_id').parquet(s_song_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Error sending http request and maximum retry encountered.


## extract columns to create artists table

In [24]:
# artists_table = df['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artists_table = df.select(
    col("artist_id").alias("artist_id"),
    col("artist_name").alias("name"),
    col("artist_location").alias("location"),
    col("artist_latitude").alias("latitude"),
    col("artist_longitude").alias("longitude"))
artists_table = artists_table.drop_duplicates(subset=['artist_id'])
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [25]:
# write songs table to parquet files partitioned by year and artist
s_s3_path = "s3a://{}/artists.parquet".format(output_data)
artists_table.write.mode('overwrite').parquet(s_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## filter by actions for song plays

In [5]:
df = spark.read.json(data_logs)
df = df.filter(df.page == 'NextSong')
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

## extract columns for users table

In [6]:
users_table = df.select(
    col("userId").alias("user_id"),
    col("firstName").alias("first_name"),
    col("lastName").alias("last_name"),
    col("gender").alias("gender"),
    col("level").alias("level"))
users_table = users_table.drop_duplicates(subset=['user_id'])
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [7]:
users_table.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     51|      Maia|    Burke|     F| free|
+-------+----------+---------+------+-----+

In [8]:
# write users table to parquet files
s_s3_path = "{}/users.parquet".format(output_data)
users_table.write.mode('overwrite').parquet(s_s3_path)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## extract columns to create time table

In [9]:
from pyspark.sql.types import DateType
from datetime import datetime
import time

# create datetime column from timestamp column
get_timestamp = udf(lambda x: time.localtime(x/1000.))
get_datetime = udf(lambda x: time.strftime(
    '%Y-%m-%d %H:%M:%S', time.localtime(x/1000.0)), DateType())

get_timestamp = udf(lambda ts: ts/1000)
get_datetime = udf(lambda ts: datetime.fromtimestamp(ts/1000), DateType())

df = df.withColumn('timestamp', get_timestamp('ts'))
df = df.withColumn('datetime', get_datetime('ts'))
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- datetime: date (nullable = true)

In [13]:
df.select('timestamp', 'datetime').limit(1).show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+
|       timestamp|  datetime|
+----------------+----------+
|1.543279932796E9|2018-11-27|
+----------------+----------+

In [11]:

from pyspark.sql.functions import (year, month, dayofmonth, hour, weekofyear,
                                   date_format, dayofweek)

# extract columns to create time table
time_table = df.select(
    col('datetime').alias('start_time'),
    hour('datetime').alias('hour'),
    dayofmonth('datetime').alias('day'),
    weekofyear('datetime').alias('week'),
    month('datetime').alias('month'),
    year('datetime').alias('year'),
    dayofweek('datetime').alias('weekday')
)
time_table = time_table.drop_duplicates(subset=['start_time'])

time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [14]:
time_table.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|2018-11-02|   0|  2|  44|   11|2018|      6|
+----------+----+---+----+-----+----+-------+

In [18]:
s_s3_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'uira-my-bucket3/songs.parquet'

In [15]:
# write time table to parquet files partitioned by year and month
s_s3_path = "{}/time.parquet".format(output_data)
time_table.write.mode('overwrite').partitionBy(
    'year', 'month').parquet(s_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## extract columns from joined song and log datasets to create songplays table

In [17]:
s_s3_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'uira-my-bucket3/songs.parquet'

In [19]:
# read in song data to use for songplays table
s_s3_path = "s3a://{}/songs.parquet".format(output_data)
songs_table = spark.read.parquet(s_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
songs_table.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------+----+------------------+
|           song_id|               title|duration|year|         artist_id|
+------------------+--------------------+--------+----+------------------+
|SOBTCUI12A8AE48B70|Faust: Ballet Mus...|94.56281|   0|ARSUVLW12454A4C8B8|
+------------------+--------------------+--------+----+------------------+

In [24]:
from pyspark.sql.functions import row_number


window = Window.orderBy(col('song_id'))

songplays_table = (df.join(
    songs_table.alias('songs'), df.song == col('songs.title'))
     .select(
        col('timestamp').alias('start_time'),
        month('datetime').alias('month'),
        year('datetime').alias('year'),
        col('userId').alias('user_id'),
        col('level').alias('level'),
        col('songs.song_id').alias('song_id'),
        col('songs.artist_id').alias('artist_id'),
        col('sessionId').alias('session_id'),
        col('location').alias('location'),
        col('userAgent').alias('user_agent'),
     )
    .withColumn('songplay_id', row_number().over(window))
)

songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- songplay_id: integer (nullable = true)

In [25]:
songplays_table.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+----+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+
|      start_time|month|year|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|songplay_id|
+----------------+-----+----+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+
|1.541786100796E9|   11|2018|     80| paid|SOAOJYY12A58A7B2F9|ARFVYJI1187B9B8E13|       416|Portland-South Po...|"Mozilla/5.0 (Mac...|          1|
+----------------+-----+----+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+

In [26]:
# write songplays table to parquet files partitioned by year and month
s_s3_path = "{}/songplays.parquet".format(output_data)
songplays_table.write.partitionBy(
    ['year', 'month']).mode('overwrite').parquet(s_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…